In [26]:
import torch
#import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
import os
from PIL import Image
from sklearn.preprocessing import LabelEncoder  # label 수로 변환
import cv2

## 참고 자료
https://www.datacamp.com/tutorial/pytorch-tutorial-building-a-simple-neural-network-from-scratch<br>
https://gaussian37.github.io/dl-pytorch-neural-network/<br>
https://learn.microsoft.com/ko-kr/windows/ai/windows-ml/tutorials/pytorch-train-model


In [2]:
# 데이터 폴더 경로
path = 'C:\\Users\\JungHyeona\\Desktop\\KNN\\image'

In [3]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img = self.images[index]
        label = self.labels[index]
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, label

In [4]:
images = []
labels = []

for for_name in os.listdir(path):  # i = ["AC", "FL", "HC", "HUM"]
    forder = os.path.join(path, for_name)  # ex) path/AC

    for j in os.listdir(forder):
        file = os.path.join(forder, j)  # ex) path/AC/a.png
        
        image = Image.open(file)
        label = os.path.basename(os.path.dirname(file))
        
        images.append(image)
        labels.append(label)

In [5]:
# label값을 숫자로 변환해주기
label_trans = {"AC" : 0, "FL" : 1, "HC" : 2, "HUM" : 3}
labels = [label_trans[label] for label in labels]

### 데이터 전처리

In [6]:
transform = transforms.Compose([
     transforms.Resize((32, 32)),
     transforms.ToTensor(),
     transforms.Normalize((-0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [7]:
# 데이터셋 인스턴스 생성
dataset = CustomDataset(images, labels, transform=transform)

In [8]:
# 데이터셋을 train, validation, test로 분리
train_dataset, test_dataset = train_test_split(dataset, test_size=0.3, random_state=42)

In [9]:
# DataLoader를 통해 데이터를 불러올 수 있도록 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

### Neural Network 모델

In [29]:
class NN(nn.Module):

    # 신경망 layer 구성요소 정의
    def __init__(self, input_size, hidden_size, output_size):
        super(NN, self).__init__()
        self.linear01 = torch.nn.Linear(input_size, hidden_size)
        self.linear02 = torch.nn.Linear(hidden_size, output_size)
        
    def forward(self,x):
        x = torch.sigmoid(self.linear01(x))
        x = self.linear02(x)
        #x = F.relu(self.linear02(x))
        return x
        
model = NN(32*32*3, 128, 4)
x = torch.randn(1, 32*32*3)
output = model(x)
# 학습하기 전의 weight와 bias를 출력합니다.
#print(list(model.parameters()))

In [30]:
print(output)

tensor([[-0.0093, -0.1852,  0.0463,  0.2739]], grad_fn=<AddmmBackward0>)


### 손실함수

In [16]:
loss_func = nn.CrossEntropyLoss()
learning_rate = 0.0002
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

In [17]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 27


In [31]:
loss_values = []
avg_cost = 0

for epoch in range(2):
    
    # 미니 배치 단위로 꺼내온다.
    for X, Y in train_loader: 
        
        # 파라미터(가중치) 0으로 초기화
        optimizer.zero_grad()
        
        # 예측과 손실 계산
        m_pred = model(X)
        loss = loss_func(m_pred, Y)
        loss_values.append(loss.item())
        
        # 역전파
        loss.backward()
        
        # 경사 갱신
        optimizer.step()
        
        avg_cost += loss / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (3072x32 and 3072x128)

### 모델 평가 함수

In [ ]:
def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [ ]:
# 모델 평가
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
val_accuracy = evaluate(model, val_loader, device)
print(f'Validation Accuracy: {val_accuracy:.2f}%')

In [ ]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [ ]:
x_train